In [1]:
from mxnet import ndarray as nd
from mxnet import autograd as ag
from mxnet import gluon

num_inputs = 2
num_examples = 1000

true_w = [1.5, 3.5]
true_b = 5.2

X = nd.random_normal(shape = (num_examples, num_inputs))
y = true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b
y += .01 * nd.random_normal(shape = y.shape)

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


Data Input

In [2]:
batch_size = 10
dataset = gluon.data.ArrayDataset(X, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle = True)

In [3]:
for data, label in data_iter:
    print(data, label)
    break


[[ 1.58492959  0.57604319]
 [-0.23258239 -0.29914257]
 [ 0.90938067 -0.83285183]
 [-0.86407942  1.45007706]
 [ 1.05059648  1.2565161 ]
 [-0.30403462  0.76864469]
 [-1.20053232  1.60998738]
 [ 0.15614767  0.40446803]
 [ 1.1579833  -0.11570764]
 [ 0.32672027  1.28878808]]
<NDArray 10x2 @cpu(0)> 
[  9.58172035   3.80227947   3.64578676   8.97133636  11.18753338
   7.4154768    9.01883888   6.84647989   6.53696489  10.18911362]
<NDArray 10 @cpu(0)>


Model

In [4]:
net = gluon.nn.Sequential()

In [5]:
net.add(gluon.nn.Dense(1))

Parameters Initialization

In [6]:
net.initialize()

Loss Function

In [7]:
square_loss = gluon.loss.L2Loss()

Optimize

In [8]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

Train

In [9]:
epochs = 5
batch_size = 10
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with ag.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
    print('Epoch %d, average loss: %f' % (e, total_loss / num_examples))        

Epoch 0, average loss: 1.098850
Epoch 1, average loss: 0.000046
Epoch 2, average loss: 0.000047
Epoch 3, average loss: 0.000047
Epoch 4, average loss: 0.000047


In [10]:
dense = net[0]
true_w, dense.weight.data()

([1.5, 3.5], 
 [[ 1.49990308  3.49974537]]
 <NDArray 1x2 @cpu(0)>)

In [11]:
true_b, dense.bias.data()

(5.2, 
 [ 5.20012522]
 <NDArray 1 @cpu(0)>)

In [12]:
help(trainer.step)

Help on method step in module mxnet.gluon.trainer:

step(batch_size, ignore_stale_grad=False) method of mxnet.gluon.trainer.Trainer instance
    Makes one step of parameter update. Should be called after
    `autograd.compute_gradient` and outside of `record()` scope.
    
    Parameters
    ----------
    batch_size : int
        Batch size of data processed. Gradient will be normalized by `1/batch_size`.
        Set this to 1 if you normalized loss manually with `loss = mean(loss)`.
    ignore_stale_grad : bool, optional, default=False
        If true, ignores Parameters with stale gradient (gradient that has not
        been updated by `backward` after last step) and skip update.



In [13]:
dense.weight??